In [ ]:
%load_ext autoreload
%autoreload 2

import logging
from pathlib import Path

import geopandas as gpd
import pandas as pd
from dotenv import dotenv_values
from shapely.geometry import LineString, Point, Polygon

from generator_drainage_units import run_generator_order_levels

logging.basicConfig(level=logging.DEBUG)

In [ ]:
# Define case and base_dir

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
waterschap = "Waterschap Vallei & Veluwe"
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"

# Define waterboard specific variables
range_orde_code_min = 721
range_orde_code_max = 760

In [ ]:
case_path = Path(base_dir, case_name)
order = run_generator_order_levels(
    path=case_path,
    waterschap=waterschap,
    range_orde_code_min=range_orde_code_min,
    range_orde_code_max=range_orde_code_max,
    create_html_map=True,
    water_lines=["hydroobjecten"],
)

In [ ]:
for col in ["upstream_edges", "downstream_edges"]:
    order.nodes[col] = order.nodes.apply(lambda x: [None if a=="" else a for a in x[col].split(',')], axis=1)
for col in ["upstream_angles", "downstream_angles"]:
    order.nodes[col] = order.nodes.apply(lambda x: [None if a=="" else float(a) for a in x[col].split(',')], axis=1)

In [ ]:
sel_one_down_one_up = order.nodes[(order.nodes.no_downstream_edges==1) & (order.nodes.no_upstream_edges==1)].index.values
sel_one_down_two_up = order.nodes[(order.nodes.no_downstream_edges==1) & (order.nodes.no_upstream_edges==2)].index.values
sel_two_down_one_up = order.nodes[(order.nodes.no_downstream_edges==2) & (order.nodes.no_upstream_edges==1)].index.values
sel_two_down_two_up = order.nodes[(order.nodes.no_downstream_edges==2) & (order.nodes.no_upstream_edges==2)].index.values

In [ ]:
from generator_drainage_units.utils.general_functions import find_edge_smallest_angle_difference, angle_difference

order.nodes['upstream_edge'] = None
order.nodes['downstream_edge'] = None
order.nodes['angle_difference'] = None

order.nodes.loc[sel_one_down_one_up, 'downstream_edge'] = order.nodes.loc[sel_one_down_one_up].apply(lambda x: x['downstream_edges'][0], axis=1)
order.nodes.loc[sel_one_down_one_up, 'upstream_edge'] = order.nodes.loc[sel_one_down_one_up].apply(lambda x: x['upstream_edges'][0], axis=1)

order.nodes.loc[sel_one_down_one_up, 'angle_difference'] = order.nodes.loc[sel_one_down_one_up].apply(lambda x: angle_difference(x['downstream_angles'][0], x['upstream_angles'][0]), axis=1)

order.nodes.loc[sel_one_down_two_up, 'downstream_edge'] = order.nodes.loc[sel_one_down_two_up].apply(lambda x: x['downstream_edges'][0], axis=1)
order.nodes.loc[sel_one_down_two_up, 'upstream_edge'] = order.nodes.loc[sel_one_down_two_up].apply(
    lambda x: find_edge_smallest_angle_difference(x['downstream_angles'][0], x['upstream_angles'], x['upstream_edges'])[1], axis=1
)

order.nodes.loc[sel_one_down_two_up, 'angle_difference'] = order.nodes.loc[sel_one_down_two_up].apply(
    lambda x: [angle_difference(a, x['downstream_angles'][0]) for a in x['upstream_angles']],
    axis=1
)

order.nodes.loc[sel_two_down_one_up, 'upstream_edge'] = order.nodes.loc[sel_two_down_one_up].apply(lambda x: x['upstream_edges'][0], axis=1)
order.nodes.loc[sel_two_down_one_up, 'downstream_edge'] = order.nodes.loc[sel_two_down_one_up].apply(
    lambda x: find_edge_smallest_angle_difference(x['upstream_angles'][0], x['downstream_angles'], x['downstream_edges'])[1], axis=1
)

order.nodes.loc[sel_two_down_one_up, 'angle_difference'] = order.nodes.loc[sel_two_down_one_up].apply(
    lambda x: [angle_difference(a, x['upstream_angles'][0]) for a in x['downstream_angles']],
    axis=1
)

order.nodes.loc[sel_two_down_two_up, 'upstream_edge'] = order.nodes.loc[sel_one_down_two_up].apply(
    lambda x: [find_edge_smallest_angle_difference(a, x['upstream_angles'], x['upstream_edges'])[1] for a in x['downstream_angles']], axis=1
)
order.nodes.loc[sel_two_down_two_up, 'downstream_edge'] = order.nodes.loc[sel_two_down_one_up].apply(
    lambda x: [find_edge_smallest_angle_difference(a, x['downstream_angles'], x['downstream_edges']) for a in x['upstream_angles']], axis=1
)

order.nodes.loc[sel_two_down_two_up, 'angle_difference'] = order.nodes.loc[sel_two_down_two_up].apply(
    lambda x: [[angle_difference(d, u) for u in x['upstream_angles']] for d in x['downstream_angles']],
    axis=1
)


In [ ]:
order.nodes.loc[sel_two_down_two_up, 'upstream_edge'] = order.nodes.loc[sel_one_down_two_up].apply(
    lambda x: [find_edge_smallest_angle_difference(a, x['upstream_angles'], x['upstream_edges'])[1] for a in x['downstream_angles']], axis=1
)
order.nodes.loc[sel_two_down_two_up, 'downstream_edge'] = order.nodes.loc[sel_two_down_one_up].apply(
    lambda x: [find_edge_smallest_angle_difference(a, x['downstream_angles'], x['downstream_edges']) for a in x['upstream_angles']], axis=1
)
order.nodes.loc[sel_two_down_two_up]

In [392]:
order.nodes.loc[sel_two_down_two_up]

,x,y,nodeID,geometry,upstream_edges,no_upstream_edges,downstream_edges,no_downstream_edges,upstream_angles,downstream_angles,upstream_edge,downstream_edge,angle_difference
226,179362.575,475981.518,226,POINT (179362.575 475981.518),"[WL_5052-1, WL_1708-19]",2,"[WL_1707-0, WL_222-0]",2,"[102.95, 99.53]","[47.6, 97.91]",NaN,NaN,"[[55.35, 51.93], [5.040000000000006, 1.6200000..."


In [ ]:
sel_two_down_two_up

In [ ]:
sel_two_down_one_up

In [ ]:
sel_one_down_one_up

In [ ]:
edges, outflow_nodes = order.generate_order_levels_to_outflow_nodes_edges()

In [ ]:
edges

In [ ]:
edges.to_file(Path(case_path, '1_tussenresultaat', 'test.gpkg'))

In [ ]:
order.nodes['downstream_edge']